In [38]:
# Libraries
import numpy as np   
import time
import scipy.sparse.linalg as spy
import warnings
warnings.filterwarnings('ignore')


def pos_def(n):
    A = np.random.rand(n,n)
    return (A+A.T) + n*np.eye(n)

n = 20
A = pos_def(n)
x_sol = np.floor(np.random.rand(n) * 100)
b = np.dot(A, x_sol)
alpha = 0.01

In [59]:
# A better implementation of preconditioned GMRes without Cauchy integral (left)
def prec_gmres(A_0,b_0,alpha,it=100,tol=1e-5,left = True):
    Q = np.zeros((b_0.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b_0.shape[0]))
    
    # copy
    A = np.copy(A_0)
    b = np.copy(b_0)
    M = A + alpha*np.identity(b.shape[0])
    
    # Left preconditioner -> A and b changes
    if left:
        b = gmres(M,b)[0]
    
    r = b - np.dot(A,x0)
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        if left:
            w = gmres(M, np.dot(A,Q[:,i]))[0]
        else:
            w = np.dot(A, gmres(M, Q[:,i])[0])
        
        for j in range(i+1):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        
        if H[i+1,i] == 0 or len(residual) == 0 or residual/beta0 < tol:
            break
    
    x_tild = np.dot(Q[:,:i+1], y)
    if left:
        return x_tild, i+1
    else:
        return gmres(M, x_tild)[0],i+1

In [63]:
def trapezoid2(myfun, N, a, b):
    x = np.linspace(0, b, N/2) # We want N bins, so N+1 points  
    h = x[1]-x[0]
    xmiddle = x[1:-1]
    int_val = 0
    for i in xmiddle:
        int_val += 2*myfun(i).real
    int_val = myfun(a) + 2*int_val + 2*myfun(0) + myfun(b)
    return 0.5*h*int_val

def z(t, c, r):
    return c + r*np.complex(np.cos(t), np.sin(t))

def dz(t, r):
    return r*np.complex(np.cos(t), np.sin(t))

def g(t,l,L,M,v,f, tol=1e-5):
    fz = f(z(t, (l+L)/2. + alpha, (L-l)/2.))
    dzz = dz(t, (L-l)/2.)
    p = fz*dzz
    gmr = p*spy.gmres(((l+L)/2. + dzz)*np.identity(v.shape[0]) - A, v)[0]
    return gmr


def cauchy_integral(l, L, alpha, v, Nf = 50,N=128):
    f = lambda x: (1. - (alpha/x)**(Nf+1))/(x - alpha)
    g1 = lambda t: g(t,l - l/2,L + L/2,alpha,v,f)
    val = trapezoid2(g1, N, -np.pi, np.pi) / (2.*np.pi)
    return val

In [64]:
# GMRes using contour integral to compute the preconditioner
def gmres(A_0, b_0, it=100, tol=1e-5, prec=False, left=True, alpha=0.0):
    Q = np.zeros((b_0.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b_0.shape[0]))
    
    A = np.copy(A_0)
    b = np.copy(b_0)
    
    if prec:
        lamb, vect = np.linalg.eig(A)
        L = np.amax(np.real(lamb))
        l = np.amin(np.real(lamb))
        d = np.amax(np.imag(lamb))
        l = alpha/2

        if left:
            b = cauchy_integral(l, L, alpha, b)
    
    r = b 
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        if prec:
            if left:
                w = cauchy_integral(l,L,alpha,np.dot(A,Q[:,i]))
            else:
                w = np.dot(A, cauchy_integral(l, L, alpha, Q[:,i]))
        else:
            w = np.dot(A,Q[:,i])
        
        for j in range(i+1):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        if H[i+1,i] == 0 or len(residual) == 0 or residual/beta0 < tol:
            break
    
    x_tild = np.dot(Q[:,:i+1], y)
    if left or not prec:
        return x_tild,i+1
    else:
        return cauchy_integral(l, L, alpha, x_tild),i+1

In [65]:
print "Exact solution:\n", x_sol
print "\nNp Solve:\n", spy.gmres(A,b)
print "\nNormal GMRes:\n", gmres(A,b)
print "\nLeft Prec NC:\n", prec_gmres(A,b,alpha)
print "\nRight Prec NC:\n", prec_gmres(A,b,alpha,left=False)
print "\nCauchy Left Prec:\n", gmres(A, b,prec=True,alpha=alpha)
print "\nCauchy Right Prec:\n", gmres(A,b,prec=True,left=False,alpha=alpha)

Exact solution:
[  9.   6.  29.  80.  26.  82.  28.  64.  73.  89.  83.  21.  79.  94.  40.
  95.  57.  30.  89.  22.]

Np Solve:
(array([  9.00012298,   5.99976557,  28.99985774,  79.99931786,
        26.00016065,  81.99996558,  28.00034468,  63.99980357,
        73.00017698,  89.00054424,  83.0004651 ,  21.0003594 ,
        78.99974479,  93.99970705,  39.99965939,  95.00038979,
        56.99964634,  29.99954007,  89.00004338,  22.00033663]), 0)

Normal GMRes:
(array([  9.00012298,   5.99976557,  28.99985774,  79.99931786,
        26.00016065,  81.99996558,  28.00034468,  63.99980357,
        73.00017698,  89.00054424,  83.0004651 ,  21.0003594 ,
        78.99974479,  93.99970705,  39.99965939,  95.00038979,
        56.99964634,  29.99954007,  89.00004338,  22.00033663]), 5)

Left Prec NC:
(array([  9.01113237,   6.01338427,  29.00802964,  79.99720605,
        26.00786683,  81.99738508,  28.00865169,  64.00003691,
        72.9991212 ,  88.99785268,  82.99931749,  21.0108826 ,
        